In [1]:
%matplotlib widget
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from src.setup_OptiProblem import system_dynamics
import time

df = pd.read_parquet("data/sym_data/sym_df_5s_res_withPower.parquet")
# df = df["2023-06-27 00:00:00":]
# df = df["2023-07-23 00:00:00":"2023-07-24 00:00:00"]

# df = df[0:10000]
timestamps = df.index.tolist()

In [2]:

zs = 2

time_hist = timestamps[0:zs]
h_hist = df["level"][0:zs].values.reshape(1,-1)
w_hist = np.vstack([df["pump1_speed"][0:zs].values,df["pump3_speed"][0:zs].values,df["pump4_speed"][0:zs].values]).reshape(3,-1)

Qout_hist = df["outflow"][0:zs].values.reshape(1,-1)

E_hist = np.vstack([df["pump1_power_est"][0:zs].values,df["pump3_power_est"][0:zs].values,df["pump4_power_est"][0:zs].values]).reshape(3,-1)

P_hist = df["pressure"][0:zs].values.reshape(1,-1)
Qin_hist = df["inflow"][0:zs].values.reshape(1,-1)


trigger_k = [1,0,0]
p = 0

K = len(df)
for k in range(2,K):

    Qout_next,E_next,P_next,h_next = system_dynamics(w_hist[:,-zs:],Qout_hist[-zs:],Qin_hist[-zs:],P_hist[-zs:],E_hist[:,-zs:],h_hist[-zs:],5)

    time_hist = np.hstack([time_hist, timestamps[k]])
    h_hist = np.hstack([h_hist, h_next.reshape(1,-1)])
    Qout_hist = np.hstack([Qout_hist, Qout_next.full()])  
    E_hist = np.hstack([E_hist, E_next.full()])   
    P_hist = np.hstack([P_hist, P_next.reshape(1,-1)])    
    Qin_hist = np.hstack([Qin_hist, df["inflow"][k].reshape(1,-1)]) 

    # ============ P-Controller ================
    # kp = 100
    # e = h_next-140
    # w_next = np.fmax(np.fmin(np.array(trigger_k)*(e*kp),1500),0)

    # ========== Fixed Efficiency-Controller ================
    kp = 100
    e = h_next-140
    w_next_calc = e*kp

    if(w_next_calc < 1500):
         w_next = np.fmax(np.array(trigger_k)*w_next_calc,0)
    elif(w_next_calc < 3000):
        w_next = np.array(trigger_k)*1500
        if(p>1):
            w_next[0] = w_next_calc - 1500
        else:
            w_next[p+1] = w_next_calc - 1500

    elif(w_next_calc < 4500):
        if(p == 2):
            w_next[0] = 1500
            w_next[1] = w_next_calc - 3000
        elif(p == 1):
            w_next[2] = 1500
            w_next[0] = w_next_calc - 3000
        elif(p == 0):
            w_next[1] = 1500
            w_next[2] = w_next_calc - 3000
    else:
        w_next = np.ones((3,1))*1500


    for i in range(3):
        if(w_next[i] < 600):
            w_next[i] = 0


    dw = (w_next - w_hist[:,-1])/5
    for i in range(3):
        if(abs(dw[i]) > 60):
            w_next[i] = w_hist[i,-1]+np.sign(dw[i])*125

    
    w_hist = np.hstack([w_hist, w_next.reshape(3,-1)])


    if ((k+100)%4000 == 0):
        p = p+1
        if (p>2): 
            p = 0
        trigger_k[p] = 1

    if (k%4000 == 0):
        if (p==0): 
            trigger_k[2] = 0
        else:
            trigger_k[p-1] = 0


    # print(f"iteration : {k+1}/{K}"),0


/tmp/ipykernel_68018/1369071807.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Qin_hist = np.hstack([Qin_hist, df["inflow"][k].reshape(1,-1)])


In [3]:
results = pd.DataFrame(list(zip(w_hist[0,:].reshape(-1), w_hist[1,:].reshape(-1), w_hist[2,:].reshape(-1), Qout_hist.reshape(-1),P_hist.reshape(-1),h_hist.reshape(-1),Qin_hist.reshape(-1),E_hist[0,:].reshape(-1), E_hist[1,:].reshape(-1), E_hist[2,:].reshape(-1))), index= time_hist,columns=df.columns)
# results["effi"] = effi_hist.reshape(-1,1)
# results.to_parquet('data/sym_data/sym_df_5s_res_P-controller_600rpm_v3.parquet')


In [4]:
from src.utils import *
FEATURES = results.columns


display(plot_features_and_target(results, FEATURES))

/home/seb/miniconda3/envs/mpc_env/lib/python3.11/site-packages/plotly_resampler/figure_resampler/utils.py:177: FutureWarning:

'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.



FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> pump1_speed <i style="color:#fc9944">~2m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '6b8c1231-3dae-403c-812c-3b9048b509b8',
              'x': array([datetime.datetime(2023, 5, 23, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2023, 5, 23, 0, 0, 20, tzinfo=<UTC>),
                          datetime.datetime(2023, 5, 23, 0, 2, 30, tzinfo=<UTC>), ...,
                          datetime.datetime(2023, 5, 24, 3, 42, 25, tzinfo=<UTC>),
                          datetime.datetime(2023, 5, 24, 3, 44, 5, tzinfo=<UTC>),
                          datetime.datetime(2023, 5, 24, 3, 46, 35, tzinfo=<UTC>)], dtype=object),
              'y': array([ 840.        , 1297.8305878 , 1210.99404732, ...,    0.        ,
                             0.        ,    0.        ])},
             {'name': '<b style="color:sandybrown">[R]</b> pump4_speed <i style